<a href="https://colab.research.google.com/github/Hetul803/finance-research-assistant/blob/main/notebooks/phi3_student_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers>=4.40.1 accelerate>=0.27.2


ERROR: Operation cancelled by user


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False,
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [4]:
from transformers import pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,
)

Device set to use cuda
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [6]:
messages = [
             {"role":"system","content":"Your are a helpful AI assisant."},
             {"role":"user","content":"Who is salman khan?"},
]
output = generator(messages)
print(output[0]["generated_text"])

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 Salman Khan is a prominent Indian actor, producer, and philanthropist. He is best known for his work in the Indian film industry, particularly in Bollywood, where he has starred in numerous successful films. Born on January 1


In [8]:
from transformers import pipeline
chatbot = pipeline("text-generation",model=model, tokenizer=tokenizer)


Device set to use cuda


In [9]:
# Default: deterministic (no sampling, so no temperature)
GEN_ARGS = {
    "max_new_tokens": 200,
    "return_full_text": False,
    "do_sample": False,
    # (no temperature/top_p keys when do_sample=False)
}

def set_sampling(on: bool, temperature: float = 0.7, top_p: float = 0.9):
    """Turn sampling on/off and set temperature/top_p without reloading the model."""
    GEN_ARGS["do_sample"] = bool(on)
    if on:
        GEN_ARGS["temperature"] = float(temperature)
        GEN_ARGS["top_p"] = float(top_p)
    else:
        # Remove keys so HF won't warn about ignored args
        GEN_ARGS.pop("temperature", None)
        GEN_ARGS.pop("top_p", None)

# Example: turn ON temperature now (adjust values anytime)
set_sampling(on=True, temperature=0.7, top_p=0.9)


In [10]:
HISTORY = []  # list of (user, assistant) tuples
SYSTEM = "You are a helpful assistant."

def chat(message: str):
    """Ask a question, keep conversation context, use current GEN_ARGS."""
    # Build messages from history
    messages = [{"role": "system", "content": SYSTEM}]
    for u, a in HISTORY:
        messages.append({"role": "user", "content": u})
        messages.append({"role": "assistant", "content": a})
    messages.append({"role": "user", "content": message})

    # Generate with the already-loaded pipeline + current GEN_ARGS
    out = chatbot(messages, **GEN_ARGS)
    answer = out[0]["generated_text"]

    # Save to history and print
    HISTORY.append((message, answer))
    print(answer)

def clear_history():
    HISTORY.clear()


In [11]:
# Temperature is ON right now from Step 2 (0.7). Ask anything:
chat("Who is Gandhi?")
chat("Explain attention very simply.")


 Mahatma Gandhi, born Mohandas Karamchand Gandhi on October 2, 1869, in Porbandar, India, was a prominent leader in India's non-violent independence movement against British rule. He was a key figure in the Indian National Congress and employed nonviolent civil disobedience tactics, which later inspired movements for civil rights and freedom across the world. Gandhi is known for his philosophy of Satyagraha, which is a form of nonviolent resistance or civil resistance. He played a pivotal role in India's struggle for independence, which was achieved in 1947. Gandhi was assassinated on January 30, 1948, by Nathuram Godse, a Hindu nationalist. His birthday, October 2, is celebrated as Gandhi Jayanti, a national holiday in India, and worldwide
 Attention is like a spotlight. When you're doing something, like reading a book or playing a game, your brain decides what to focus on. This spotlight helps you ignore other things around you that you don't need to think about right now. It's like 

In [13]:
chat("Explain algorithms.")



 An algorithm is a set of step-by-step instructions for solving a problem or accomplishing a task. Think of it as a recipe that tells you exactly what to do. For example, when you want to bake a cake, you follow a recipe that lists ingredients and directions, like mixing flour, sugar, and eggs, then baking them in the oven. In computer science, algorithms are used by computers to perform calculations, make decisions, and process data. They can range from simple ones, like sorting numbers from smallest to largest, to complex ones, like creating pictures with computer programs.


In [14]:
chat("Explain quantum computing.")

 Quantum computing is a type of computing that uses the principles of quantum mechanics, which is the science that describes how tiny particles like atoms and photons behave. Unlike traditional computers that use bits to store information as 0s or 1s, quantum computers use quantum bits, or qubits, that can be in a state of 0, 1, or both at the same time, thanks to a property called superposition. This allows quantum computers to process a vast amount of possibilities all at once. Another important principle is entanglement, where qubits can be linked in such a way that the state of one (whether it's 0 or 1) can depend on the state of another, no matter how far apart they are. This can lead to faster processing speeds and the ability to solve certain complex problems much more efficiently than classical computers.
